In [38]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

In [39]:
# Чтение статических данных из Excel файла
static_data = pd.read_excel('StaticData.xlsx')

In [40]:
# Выделение статических признаков
static_features = static_data[['X:I1', 'Y:I1', 'poro:I1', 'perm:I1',
                               'X:I2', 'Y:I2', 'poro:I2', 'perm:I2',
                               'X:I3', 'Y:I3', 'poro:I3', 'perm:I3',
                               'X:P1', 'Y:P1', 'poro:P1', 'perm:P1',
                               'X:P2', 'Y:P2', 'poro:P2', 'perm:P2',
                               'X:P3', 'Y:P3', 'poro:P3', 'perm:P3']]


In [29]:
# Нормализация статических признаков
static_scaler = StandardScaler()
static_features = static_scaler.fit_transform(static_features)


In [41]:
# Чтение динамических данных из Excel файла
dynamic_data = pd.read_excel('DynamicData_for_prediction.xlsx')


In [42]:
# Выделение динамических признаков
dynamic_features = dynamic_data[['WBHP:I1', 'WWIR:I1', 'WBHP:I2', 'WWIR:I2', 'WBHP:I3', 'WWIR:I3',
                                 'WBHP:P1', 'WLPR:P1', 'WWCT:P1', 'WOPR:P1',
                                 'WBHP:P2', 'WLPR:P2', 'WWCT:P2', 'WOPR:P2',
                                 'WBHP:P3', 'WLPR:P3', 'WWCT:P3', 'WOPR:P3']]


In [43]:
# Нормализация динамических признаков
dynamic_scaler = StandardScaler()
dynamic_features = dynamic_scaler.fit_transform(dynamic_features)


In [44]:
# Выделение координат скважин
coordinates = static_data[['X:I1', 'Y:I1', 'X:I2', 'Y:I2', 'X:I3', 'Y:I3', 'X:P1', 'Y:P1', 'X:P2', 'Y:P2', 'X:P3', 'Y:P3']]


In [54]:
print(coordinates.iloc[0])

X:I1    1150
Y:I1    1450
X:I2    1850
Y:I2     650
X:I3     660
Y:I3     450
X:P1    1550
Y:P1    1050
X:P2    1250
Y:P2     650
X:P3     760
Y:P3     950
Name: 0, dtype: int64


In [55]:

# Вычисление матрицы расстояний между скважинами
num_nodes = coordinates.shape[0]
distances = np.zeros((num_nodes, num_nodes))

for i in range(1, num_nodes):
    for j in range(1, num_nodes):
        dist = np.linalg.norm(coordinates.iloc[i] - coordinates.iloc[j])  # Используем только первую строку координат
        distances[i, j] = dist

print(distances)

[[0.]]


In [51]:
print(coordinates.head())

   X:I1  Y:I1  X:I2  Y:I2  X:I3  Y:I3  X:P1  Y:P1  X:P2  Y:P2  X:P3  Y:P3
0  1150  1450  1850   650   660   450  1550  1050  1250   650   760   950


In [33]:
# Создание графовой структуры
num_nodes = static_features.shape[0]
edges = []
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        dist = np.linalg.norm(static_features[i, :2] - static_features[j, :2])  # Расстояние между скважинами по координатам
        edges.append([i, j, 1.0 / dist])  # Используем обратное расстояние в качестве веса ребра


In [37]:
print(dist)

dist


In [34]:
# Преобразование списка ребер в тензор PyTorch
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()


In [35]:
if len(edges) > 0:
    data = Data(x=torch.tensor(static_features, dtype=torch.float32),
                edge_index=edges[:2],  # Первые два элемента тензора edges представляют индексы начала и конца ребра
                edge_attr=edges[2].unsqueeze(1),  # Третий элемент тензора edges представляет вес ребра (расстояние)
                y=torch.tensor(dynamic_features, dtype=torch.float32))
else:
    print("No edges found. Please check your data.")


No edges found. Please check your data.
